## 从key文件读取key值
*输入参数: file_path
文件路径
*返回值: 存有key的列表

In [1]:
def read_keys_from_file(key_path):
    key_pool = []
    with open(key_path, 'r') as file:
        for line in file:
            key = line.strip()  # 去掉每一行的换行符
            key_pool.append(key)  # 将每一行存入key池中
    return key_pool

In [2]:
# 读取key
key_path = "key.txt"  # 将这个替换为你的txt文件的路径
key_pool = read_keys_from_file(key_path)

In [3]:
import os
from random import random
os.environ['HTTP_PROXY'] = "127.0.0.1:33210"
os.environ['HTTPS_PROXY'] = "127.0.0.1:33210"
os.environ["OPENAI_API_KEY"] = key_pool[1]

from langchain.document_loaders import PyPDFLoader

from PyPDF2 import PdfMerger
file_list = ["KT820说明书V2.0 201909.pdf","GSK980TC3系列  编程操作手册（2018年6月第3版）0622定(1).pdf"]
file_merger = PdfMerger()       # 合并pdf文件
for pdf in file_list:
    file_merger.append(pdf)
# 合并后的pdf放在本地
file_merger.write("merge.pdf")
loader = PyPDFLoader("merge.pdf")
unprocessed_documents = loader.load()


In [76]:
# 用户输入的问题
question = "请问文档里面的kt820和GSK980TC3有什么区别"

import jieba
# 利用jieba_fast进行分词
def jieba_extract_keywords(question):
    # 使用jieba进行分词
    keywords = jieba.cut(question)
    return keywords

# 利用chatgpt进行分词
def gpt_extract_keywords(question):

    from langchain.prompts import ChatPromptTemplate
    prompts = f"""
        ----
        {question}
        ----
        注意如果question是数学表达式或包含数学表达式，不需要提取数学表达式中的关键词而是直接返回完整的数学表达式。
        你是一个专家级的文本关键词提取大师，根据上面的question，从其中提取关键词并按行地返回它们。关键词提取应该尽可能全面，包括单个有意义的词汇和有意义的词组。
        特别强调question中出现名词后跟着数字的情况，比如“报警号199”应拆分为“报警号”、“199”和“报警号199”。
        只需要输出关键词，只需要输出关键词！
        如果提取关键词为空，直接返回{question}！

        例如：question = 100+100
        回答：
        100+100

    """


    promptsTemplate = ChatPromptTemplate.from_template(prompts)
    from langchain.chains import LLMChain
    from langchain.chat_models import ChatOpenAI
    llm = ChatOpenAI(model_name='gpt-3.5-turbo-16k', temperature=0)
    chain = LLMChain(llm=llm, prompt=promptsTemplate)
    response = chain.run(question=question)
    return response

In [77]:
import PyPDF2
def search_keywords_in_pdf(doc,keywords):
    keyword_count_per_page = {}
    for page_num in range(len(doc)):
        # 计算当前页面的关键词出现次数
        count = sum(doc[page_num].page_content.count(keyword) for keyword in keywords)

        # 如果关键词出现次数大于0，则记录该页码和次数
        if count > 0:
            keyword_count_per_page[page_num+1] = count
    # 根据关键词出现次数从大到小排序页面号
    sorted_page_numbers = [k for k, v in sorted(keyword_count_per_page.items(), key=lambda item: item[1],reverse=True)]
    return sorted_page_numbers

def get_page_content(file_path, page_number):
    with open(file_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)

        # 确保页面号在有效范围内
        if page_number - 1 < len(reader.pages):
            page = reader.pages[page_number - 1]
            text = page.extract_text()
            return text
        else:
            return "指定的页码超出了PDF文件的范围"


In [78]:
import time

start_time = time.time()        # 记录开始时间
# 从问题中提取关键词
keywords = gpt_extract_keywords(question)
print(keywords)
import ast
keywords = keywords.splitlines()

print(keywords)
print(type(keywords))

kt820, GSK980TC3
['kt820, GSK980TC3']
<class 'list'>


In [79]:

os.environ["OPENAI_API_KEY"] = key_pool[2]
from langchain.chat_models import ChatOpenAI


In [80]:
pages = search_keywords_in_pdf(unprocessed_documents,keywords)
print(f"关键词匹配的页面有: {pages}")

关键词匹配的页面有: []


In [81]:
documents = []
# 控制页面的数量
pages = pages[:30]

from collections import defaultdict
# 使用 defaultdict 存储未处理的文档，键为页面编号，值为文档列表
unprocessed_documents_dict = defaultdict(list)
for doc in unprocessed_documents:
    unprocessed_documents_dict[doc.metadata['page']].append(doc)

# 使用列表推导式来按页面顺序添加文档
documents = [doc for page in pages for doc in unprocessed_documents_dict[page - 1] if page - 1 in unprocessed_documents_dict]

print(documents)
print(len(documents))

[]
0


In [82]:
from langchain.text_splitter import CharacterTextSplitter
os.environ["OPENAI_API_KEY"] = key_pool[3]


text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

from langchain.vectorstores import Chroma

db = Chroma.from_documents(texts, embeddings)
retriever = db.as_retriever()

from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-0613"), chain_type="stuff",
                                 retriever=retriever,
                                 return_source_documents=True)

prompt = f"""
    下面是用户提的问题,以'''为分割符号
    '''
    {question}
    '''
    如果问题是数学表达式，不用参考文本，直接计算结果。
    回答要求语气温和有礼貌、并且回答内容要详细、有具体文本支撑观点。
    """

res = qa(prompt)
answer, docs = res['result'], res['source_documents']
end_time = time.time()
execution_time = end_time - start_time
print(f"代码运行时间: {execution_time} 秒")
print(answer)


代码运行时间: 21.98273801803589 秒
根据提供的文本，我们可以得出以下信息：

1. KT-820TI数控车床系统和GSK980TC3系列总线式车床数控系统是两个不同的数控系统。

2. KT-820TI数控车床系统的编程操作手册包含了关于编程基础、G功能、螺纹功能、补偿功能、刀具功能、主轴功能、辅助功能、程序编辑和操作功能等方面的内容。

3. GSK980TC3系列总线式车床数控系统的编程操作手册包含了关于编程说明、系统通信、技术规格、运动控制功能、显示界面、G功能、螺纹功能、补偿功能、刀具功能、主轴功能、辅助功能、程序编辑和操作功能等方面的内容。

根据以上信息，KT-820TI数控车床系统和GSK980TC3系列总线式车床数控系统在功能和内容上可能存在一些差异。具体的区别可能需要进一步的比较和分析。
